In [6]:
import os, sys
sys.path.append(os.path.join(os.getcwd(), "../keras-deep-graph-learning")) # Adding the submodule to the module search path
sys.path.append(os.path.join(os.getcwd(), "../keras-deep-graph-learning/examples")) # Adding the submodule to the module search path

import numpy as np
from keras.layers import Dense, Activation, Dropout
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import Adam
from keras_dgl.layers import GraphCNN
import keras.backend as K
from keras.utils import to_categorical

import networkx as nx
import pandas as pd

In [7]:
print("Creating our simple sample data...")
A = np.array([[0,1,5], [1,0,0], [5,0,0]])
print(A)
X = np.array([[1,2,10], [4,3,10], [0,2,11]]) # features, whatever we have there...

# Notice, if we set A = identity matrix, then we'd effectively assume no edges and just do a basic
# MLP on the features.

# We could do the same by setting the graph_conv_filter below to Id.

# We could also set X to Id, and thus effectively assume no features, and in this way
# do an "edge" embedding, so effectively try to understand what's connected to what.

# We could then use that as feature in any way we like...

Y_o_dim = np.array([1,2,1])
Y =  to_categorical(Y_o_dim) # labels, whatever we wanna classify things into... in categorical form.

Creating our simple sample data...
[[0 1 5]
 [1 0 0]
 [5 0 0]]


In [8]:
train_on_weight= np.array([1,1,0])
print("Now we won't do any fancy preprocessing, just basic training.")

NUM_FILTERS = 1
graph_conv_filters =  A # you may try np.eye(3)
graph_conv_filters = K.constant(graph_conv_filters)

model = Sequential()
model.add(GraphCNN(Y.shape[1], NUM_FILTERS, graph_conv_filters, input_shape=(X.shape[1],), activation='elu', kernel_regularizer=l2(5e-4)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), metrics=['acc'])
model.summary()

model.fit(X, Y, batch_size=A.shape[0], sample_weight=train_on_weight, epochs=200, shuffle=False, verbose=0)
Y_pred = model.predict(X, batch_size=A.shape[0])
print(np.argmax(Y_pred, axis=1))

Now we won't do any fancy preprocessing, just basic training.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
graph_cnn (GraphCNN)         (3, 3)                    12        
_________________________________________________________________
activation (Activation)      (3, 3)                    0         
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________
[1 2 1]


In [9]:

data_dir = os.path.expanduser("C:/Users/caell/data_science/data_science_projects/keras-deep-graph-learning/data/cora")
edgelist = pd.read_csv(os.path.join(data_dir, "cora.cites"), sep='\t', header=None, names=["target", "source"])
edgelist["label"] = "cites"

feature_names = ["w_{}".format(ii) for ii in range(1433)]
column_names =  feature_names + ["subject"]
node_data = pd.read_csv(os.path.join(data_dir, "cora.content"), sep='\t', header=None, names=column_names)


In [10]:
edgelist

,target,source,label
0,35,1033,cites
1,35,103482,cites
2,35,103515,cites
3,35,1050679,cites
4,35,1103960,cites
...,...,...,...
5424,853116,19621,cites
5425,853116,853155,cites
5426,853118,1140289,cites
5427,853155,853118,cites


In [11]:
node_data

,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,...,w_1424,w_1425,w_1426,w_1427,w_1428,w_1429,w_1430,w_1431,w_1432,subject
31336,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Neural_Networks
1061127,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,Rule_Learning
1106406,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
13195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
37879,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Probabilistic_Methods
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128975,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Genetic_Algorithms
1128977,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Genetic_Algorithms
1128978,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Genetic_Algorithms
117328,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Case_Based
